<a href="https://colab.research.google.com/github/marialago/AbstractArtWithDCGAN/blob/main/ArtWithDCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Bloco 1: Importações e Configuração
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time
import os

# --- CONFIGURAÇÃO ---
IMAGE_SIZE = 64
IMAGE_CHANNELS = 3
BATCH_SIZE = 128
BUFFER_SIZE = 8000
NOISE_DIM = 256
EPOCHS = 1000
GEN_LR = 2e-4
DISC_LR = 1e-4
ADAM_BETA_1 = 0.5
OUTPUT_DIR = 'generated_images_new_dataset' # Diretório para salvar imagens geradas

# Cria o diretório de saída se ele não existir
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

print("Configurações carregadas.")
print(f"TensorFlow Version: {tf.__version__}")


In [ ]:
# Bloco 2: Carregamento e Pré-processamento de Dados (Dataset do Kaggle)
import tensorflow as tf

# dataset_dir_path = 'kaggle_dataset/abstract_patterns'
dataset_dir_path = 'kaggle_dataset/img' # Ajuste conforme necessário!

def load_kaggle_dataset():
    print(f"Carregando imagens de: {dataset_dir_path}")

    # Verifica se o diretório existe
    if not os.path.exists(dataset_dir_path) or not os.listdir(dataset_dir_path):
         raise ValueError(f"Diretório do dataset '{dataset_dir_path}' não encontrado ou vazio. "
                          "Verifique o caminho e se o download/descompactação funcionou.")


    # labels=None e label_mode=None porque GANs são não supervisionadas
    # shuffle=False aqui, vamos embaralhar depois da normalização
    # batch_size=None para carregar tudo e depois aplicar map/batch, OU defina BATCH_SIZE direto
    train_dataset = tf.keras.utils.image_dataset_from_directory(
        dataset_dir_path,
        labels=None, # Não precisamos de rótulos para GAN
        label_mode=None,
        image_size=(IMAGE_SIZE, IMAGE_SIZE),
        interpolation='nearest', # Ou 'bilinear'
        batch_size=BATCH_SIZE, # Processa em lotes
        shuffle=True # Pode embaralhar aqui ou depois
    )

    # Função para normalizar as imagens para o intervalo [-1, 1]
    def normalize_img(image):
        # As imagens carregadas por image_dataset_from_directory estão em [0, 255]
        image = tf.cast(image, tf.float32)
        image = (image - 127.5) / 127.5
        return image

    # Aplica a normalização a cada imagem no dataset
    train_dataset = train_dataset.map(normalize_img,
                                      num_parallel_calls=tf.data.AUTOTUNE)

    # Embaralha e otimiza o carregamento (prefetch)
    # Se shuffle=True em image_dataset_from_directory, este shuffle pode ser redundante mas não prejudica
    train_dataset = train_dataset.shuffle(BUFFER_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

    print("Dataset do Kaggle carregado e pré-processado.")
    # Verifica um lote de exemplo (opcional)
    for image_batch in train_dataset.take(1):
        print("Forma do lote de imagens:", image_batch.shape)
        print("Valores min/max:", tf.reduce_min(image_batch).numpy(), tf.reduce_max(image_batch).numpy())

    return train_dataset

# Carrega o dataset (substitui a chamada anterior)
train_dataset = load_kaggle_dataset()


In [ ]:
# Bloco 3: Modelo Gerador (Robusto - CORRIGIDO)
import tensorflow as tf
from tensorflow.keras import layers

# (Certifique-se que NOISE_DIM, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS estão definidos)

def make_robust_generator_model_corrected():
    model = tf.keras.Sequential(name="Robust_Generator_Corrected")

    # Ponto de partida: Camada Densa e Reshape
    start_h, start_w = IMAGE_SIZE // 8, IMAGE_SIZE // 8 # 8x8
    model.add(layers.Dense(start_h * start_w * 1024, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((start_h, start_w, 1024)))
    # Saída: (None, 8, 8, 1024)

    # Bloco de Upsampling 1: 8x8 -> 16x16
    model.add(layers.Conv2DTranspose(512, 5, strides=(2, 2), padding='same', use_bias=False))
    # Saída: (None, 16, 16, 512)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # Bloco de Upsampling 2: 16x16 -> 32x32
    model.add(layers.Conv2DTranspose(256, 5, strides=(2, 2), padding='same', use_bias=False))
    # Saída: (None, 32, 32, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # Bloco de Upsampling 3 (Final): 32x32 -> 64x64
    # A camada final usa 'tanh' para mapear para [-1, 1]
    model.add(layers.Conv2DTranspose(IMAGE_CHANNELS, 5, strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    # Saída: (None, 64, 64, IMG_CHANNELS)

    # Verifica a forma final
    assert model.output_shape == (None, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)

    return model

# Cria o gerador robusto corrigido
generator = make_robust_generator_model_corrected() # Use esta função
print("Modelo Gerador Robusto (Corrigido) criado.")
generator.summary()


In [ ]:
# Bloco 4: Modelo Discriminador (Modificado - Kernel 4x4)
import tensorflow as tf
from tensorflow.keras import layers

def make_modified_discriminator_model():
    model = tf.keras.Sequential(name="Discriminator_SN_k4x4", layers=[ # Nome atualizado
        layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)),

        # Bloco 1: 64x64 -> 32x32
        # *** KERNEL 5x5 ***
        layers.SpectralNormalization(
            layers.Conv2D(64, 5, strides=(2, 2), padding='same')
        ),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        # Bloco 2: 32x32 -> 16x16
        # *** KERNEL 5x5 ***
        layers.SpectralNormalization(
            layers.Conv2D(128, 5, strides=(2, 2), padding='same')
        ),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        # Bloco 3: 16x16 -> 8x8
        # *** KERNEL 5x5 ***
        layers.SpectralNormalization(
            layers.Conv2D(256, 5, strides=(2, 2), padding='same')
        ),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        # Bloco 4: 8x8 -> 4x4
        # *** KERNEL 5x5 ***
        layers.SpectralNormalization(
            layers.Conv2D(512, 5, strides=(2, 2), padding='same')
        ),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.4),

        # Final: Flatten e Dense(1) para logit
        layers.Flatten(),
        layers.SpectralNormalization(
            layers.Dense(1)
        )
    ])
    return model

# Cria o discriminador modificado
discriminator = make_modified_discriminator_model() # Substitui o discriminador anterior
print("Modelo Discriminador Modificado (Kernel 4x4, SN nativa) criado.")
discriminator.summary()


In [ ]:
# Bloco 5: Funções de Perda e Otimizadores

# Função de perda de entropia cruzada binária (adequada para classificação binária)
# from_logits=True porque a saída do discriminador não tem ativação sigmoid
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output, smooth_factor=0.9): # Adiciona smooth_factor
    # Label Smoothing: rótulo para reais é 0.9 em vez de 1.0
    smooth_factor = 0.9
    real_loss = cross_entropy(tf.ones_like(real_output) * smooth_factor, real_output)
    # Rótulo para falsos continua sendo 0.0
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
     # O gerador ainda quer que o discriminador pense que as fakes são reais (rótulo 1.0)
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Otimizadores Adam para ajustar os pesos dos modelos
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=GEN_LR, beta_1=ADAM_BETA_1)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=DISC_LR, beta_1=ADAM_BETA_1)

print("Funções de perda e otimizadores definidos.")


In [ ]:
# Bloco 6: Checkpointing (salvar o progresso)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

print(f"Checkpoint será salvo em: {checkpoint_dir}")
# Para restaurar o último checkpoint (se existir):
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
print("Checkpoint restaurado (se encontrado).")


In [ ]:
# Bloco 7: Definição do Passo de Treinamento

# @tf.function compila a função para um grafo TensorFlow, o que acelera a execução
@tf.function
def train_step(images):
    # Gera um lote de vetores de ruído aleatório
    noise = tf.random.normal([tf.shape(images)[0], NOISE_DIM])

    # Usa GradientTape para registrar as operações para diferenciação automática
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Gera imagens falsas a partir do ruído
        generated_images = generator(noise, training=True) # training=True ativa camadas como BatchNormalization e Dropout

        # Obtém as previsões do discriminador para imagens reais e falsas
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Calcula as perdas do gerador e do discriminador
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Calcula os gradientes das perdas em relação às variáveis treináveis dos modelos
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Aplica os gradientes aos otimizadores para atualizar os pesos dos modelos
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    # Retorna as perdas para monitoramento
    return gen_loss, disc_loss

print("Função de passo de treinamento definida.")


In [ ]:
# Bloco 8: Função para Gerar e Salvar Imagens

seed = tf.random.normal([16, NOISE_DIM]) # Gera 16 imagens para visualização

def generate_and_save_images(model, epoch, test_input):
   # `training=False` para modo de inferência (importante para BN e Dropout, se houver)
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4)) # Tamanho padrão 4x4

    for i in range(predictions.shape[0]): # predictions.shape[0] == 16
        plt.subplot(4, 4, i + 1)
        # Denormaliza a imagem de [-1, 1] para [0, 1]
        img_to_show = (predictions[i] + 1) / 2.0
        plt.imshow(img_to_show)
        plt.axis('off')

    # Salva a figura no diretório de saída
    plt.savefig(os.path.join(OUTPUT_DIR, 'image_at_epoch_{:04d}.png'.format(epoch)))
    plt.close(fig) # Fecha a figura para liberar memória
    # plt.show() # Opcional: exibe o plot diretamente no Colab (pode poluir a saída)

print("Função para gerar e salvar imagens definida.")


In [ ]:
# Bloco 9: Função do Loop de Treinamento

def train(dataset, epochs):
    print("Iniciando o treinamento...")
    gen_losses_history = []
    disc_losses_history = []

    for epoch in range(epochs):
        start_time = time.time() # Marca o tempo de início da época
        gen_loss_epoch = []      # Lista para armazenar perdas do gerador na época
        disc_loss_epoch = []     # Lista para armazenar perdas do discriminador na época

        # Itera sobre os lotes do dataset
        for batch_num, image_batch in enumerate(dataset):
            g_loss, d_loss = train_step(image_batch) # Executa um passo de treinamento
            gen_loss_epoch.append(g_loss.numpy())    # Armazena as perdas
            disc_loss_epoch.append(d_loss.numpy())


        # Gera e salva imagens de exemplo no final de cada época
        generate_and_save_images(generator, epoch + 1, seed)

        # Salva o modelo (checkpoint) a cada 15 épocas
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
            print(f"Checkpoint salvo para a época {epoch + 1}")

        # Calcula a média das perdas da época
        avg_gen_loss = np.mean(gen_loss_epoch)
        avg_disc_loss = np.mean(disc_loss_epoch)
        gen_losses_history.append(avg_gen_loss)
        disc_losses_history.append(avg_disc_loss)

        epoch_time = time.time() - start_time # Calcula o tempo da época

        # Imprime o resumo da época
        print(f'Época {epoch + 1}/{epochs} concluída em {epoch_time:.2f} seg')
        print(f'  Perda Média Gerador: {avg_gen_loss:.4f}, Perda Média Discriminador: {avg_disc_loss:.4f}')

    # Gera uma imagem final após a última época
    generate_and_save_images(generator, epochs, seed)
    print("Treinamento concluído.")
    plt.figure(figsize=(12, 6))
    plt.plot(gen_losses_history, label='Perda Gerador')
    plt.plot(disc_losses_history, label='Perda Discriminador')
    plt.title("Evolução das Perdas Durante o Treinamento")
    plt.xlabel("Épocas")
    plt.ylabel("Perda (Binary Cross-Entropy)")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(OUTPUT_DIR, "loss_plot.png"))
    plt.show()

print("Função de treinamento definida.")


In [ ]:
# Bloco 10: Executar o Treinamento
train(train_dataset, EPOCHS)

In [ ]:
# Bloco 11: Gerar uma Imagem Após o Treinamento

num_samples = 32
grid_size = int(num_samples**0.5) # Tamanho da grade para exibir as imagens

sample_noise = tf.random.normal([num_samples, NOISE_DIM])
generated_samples = generator(sample_noise, training=False)

fig, axes = plt.subplots(grid_size, grid_size, figsize=(grid_size, grid_size))

for i, ax in enumerate(axes.flatten()):
    img = (generated_samples[i] * 127.5 + 127.5).numpy().astype("uint8")
    ax.imshow(img)
    ax.axis("off")

plt.tight_layout()
plt.show()
